## Объединение наборов данных: слияние и соединение

In [1]:
import numpy as np
import pandas as pd

    Реляционная алгебра (relation algebra) - формальный набор правил манипулирования реляционными данными, формирующий теоретические основания для имеющихся в большинстве баз данных операций.
    Основной интерфейс для них - ф-ия pd.merge()

### Виды соединений
    pd.merge() реализует множество типов соединений:
    "один-к-одному"
    "многие-к-одному"
    "многие-ко-многим"

#### Соединения "один-к-одному"

In [2]:
df1 = pd.DataFrame(
    {'employee': ['Bob', 'Jake', 'Lisa', 'Sue'],
     'group': ['Accounting', 'Engineering', 'Engineering', 'HR']}
)
df2 = pd.DataFrame(
    {'employee': ['Lisa', 'Bob', 'Jake', 'Sue'],
     'hire_date': [2004, 2008, 2012, 2014]}
)
df1

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR


In [3]:
df2

,employee,hire_date
0,Lisa,2004
1,Bob,2008
2,Jake,2012
3,Sue,2014


In [4]:
# теперь объединими эту инфу
df3 = pd.merge(df1, df2)

In [5]:
df3
# ф-ия megrge() распознает, что в обоих DataFrame существует столбец employee и автоматически использует его в качестве ключа

,employee,group,hire_date
0,Bob,Accounting,2008
1,Jake,Engineering,2012
2,Lisa,Engineering,2004
3,Sue,HR,2014


#### Соединения "многие-к-одному"

In [6]:
df4 = pd.DataFrame(
    {'group': ['Accounting', 'Engineering', 'HR'],
     'supervisor': ['Carly', 'Guido', 'Steve']}
)
df4

,group,supervisor
0,Accounting,Carly
1,Engineering,Guido
2,HR,Steve


In [7]:
pd.merge(df3, df4)
# соединение прошло по group

,employee,group,hire_date,supervisor
0,Bob,Accounting,2008,Carly
1,Jake,Engineering,2012,Guido
2,Lisa,Engineering,2004,Guido
3,Sue,HR,2014,Steve


#### Соединения "многие-ко-многим"

In [8]:
df5 = pd.DataFrame({'group': ['Accounting', 'Accounting', 'Engineering', 'Engineering', 'HR', 'HR'],
                    'skills': ['math', 'spreadsheets', 'coding', 'linux', 'spreadsheets', 'organization']})
df5

,group,skills
0,Accounting,math
1,Accounting,spreadsheets
2,Engineering,coding
3,Engineering,linux
4,HR,spreadsheets
5,HR,organization


In [9]:
pd.merge(df1, df5)

,employee,group,skills
0,Bob,Accounting,math
1,Bob,Accounting,spreadsheets
2,Jake,Engineering,coding
3,Jake,Engineering,linux
4,Lisa,Engineering,coding
5,Lisa,Engineering,linux
6,Sue,HR,spreadsheets
7,Sue,HR,organization


### Задание ключа слияния
    Часто имена столбцоа не совпадают

#### Ключевое слово - on
    Проще всего указать название ключевого столбца используя параметр on, передав ему название или список.
    on - работает только в случае, когда в обоих DataFrame имеется указанное название столбца.

In [10]:
pd.merge(df1, df2, on='employee')

,employee,group,hire_date
0,Bob,Accounting,2008
1,Jake,Engineering,2012
2,Lisa,Engineering,2004
3,Sue,HR,2014


#### Ключевые слова left_on и right_on
    Для выполнения слияний наборов с разными именами столбцов

In [11]:
df3 = pd.DataFrame(
    {'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
     'salary': [70000, 80000, 120000, 90000]}
)
df3

,name,salary
0,Bob,70000
1,Jake,80000
2,Lisa,120000
3,Sue,90000


In [12]:
df1

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR


In [13]:
pd.merge(df1, df3, left_on='employee', right_on='name')

,employee,group,name,salary
0,Bob,Accounting,Bob,70000
1,Jake,Engineering,Jake,80000
2,Lisa,Engineering,Lisa,120000
3,Sue,HR,Sue,90000


In [15]:
# избыточный столбец можно удалить при помощи метода объекта DataFrame .drop()
pd.merge(df1, df3, left_on='employee', right_on='name').drop('name', axis=1)

,employee,group,salary
0,Bob,Accounting,70000
1,Jake,Engineering,80000
2,Lisa,Engineering,120000
3,Sue,HR,90000


#### Ключевые слова left_index и right_index
    Иногда удобнее вместо слияния по столбцу выполнить слияние по индексу

In [17]:
df1a = df1.set_index('employee')
df1a

,group
employee,
Bob,Accounting
Jake,Engineering
Lisa,Engineering
Sue,HR


In [18]:
df2a = df2.set_index('employee')
df2a

,hire_date
employee,
Lisa,2004
Bob,2008
Jake,2012
Sue,2014


In [19]:
pd.merge(df1a, df2a, left_index=True, right_index=True)

,group,hire_date
employee,,
Bob,Accounting,2008
Jake,Engineering,2012
Lisa,Engineering,2004
Sue,HR,2014


In [20]:
# для удобства в DataFrame реализован метод .join(), выполняющий слияние по индексам по умолчанию
df1a.join(df2a)

,group,hire_date
employee,,
Bob,Accounting,2008
Jake,Engineering,2012
Lisa,Engineering,2004
Sue,HR,2014


In [21]:
# возможна комбинация слияния по столбцам и индексам:
df1a

,group
employee,
Bob,Accounting
Jake,Engineering
Lisa,Engineering
Sue,HR


In [22]:
df3

,name,salary
0,Bob,70000
1,Jake,80000
2,Lisa,120000
3,Sue,90000


In [23]:
pd.merge(df1a, df3, left_index=True, right_on='name')

,group,name,salary
0,Accounting,Bob,70000
1,Engineering,Jake,80000
2,Engineering,Lisa,120000
3,HR,Sue,90000


### Задание операций над множествами для соединений
    Случаи когда не все ключи совпадают

In [24]:
df6 = pd.DataFrame({'name': ['Peter', 'Paul', 'Mary'],
                    'food': ['fish', 'beans', 'bread']},
                  columns=['name', 'food'])
df6

,name,food
0,Peter,fish
1,Paul,beans
2,Mary,bread


In [25]:
df7 = pd.DataFrame({'name': ['Mary', 'Joseph'],
                    'drink': ['wine', 'beer']},
                  columns=['name', 'drink'])
df7

,name,drink
0,Mary,wine
1,Joseph,beer


In [26]:
pd.merge(df6, df7)
# по умолчанию результат содержит только пересечение 2х вхоных множеств (inner join)

,name,food,drink
0,Mary,bread,wine


In [27]:
# можно задать явно
pd.merge(df6, df7, how='inner')

,name,food,drink
0,Mary,bread,wine


In [ ]:
# how = 'outer', 'left', 'right', 'inner'

In [28]:
pd.merge(df6, df7, how='outer')
# внешнее соединение - объединение всех входящих столбцов

,name,food,drink
0,Peter,fish,NaN
1,Paul,beans,NaN
2,Mary,bread,wine
3,Joseph,NaN,beer


In [29]:
# по левой табличке left
pd.merge(df6, df7, how='left')

,name,food,drink
0,Peter,fish,NaN
1,Paul,beans,NaN
2,Mary,bread,wine


In [30]:
# по правой табличке right
pd.merge(df6, df7, how='right')

,name,food,drink
0,Mary,bread,wine
1,Joseph,NaN,beer


### Пересекающиеся названия столбцов:
### Ключевое слово suffixes
    Случай, когда в двух входных объектах присутствуют конфликтующие названия столбцов.

In [31]:
df8 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'rank': [1, 2, 3, 4]})
df9 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'rank': [3, 1, 4, 2]})
df8

,name,rank
0,Bob,1
1,Jake,2
2,Lisa,3
3,Sue,4


In [32]:
df9

,name,rank
0,Bob,3
1,Jake,1
2,Lisa,4
3,Sue,2


In [33]:
pd.merge(df8, df9)

,name,rank


In [34]:
pd.merge(df8, df9, on='name')
# ф-ия слияния автоматически добавила в названия суффиксы _x _y

,name,rank_x,rank_y
0,Bob,1,3
1,Jake,2,1
2,Lisa,3,4
3,Sue,4,2


In [35]:
# если мы хотим свои суффиксы
pd.merge(df8, df9, on='name', suffixes=['_Low', '_Uppers_lalala'])

,name,rank_Low,rank_Uppers_lalala
0,Bob,1,3
1,Jake,2,1
2,Lisa,3,4
3,Sue,4,2


### Пример: Данные по штатам США

In [36]:
pop = pd.read_csv('data/state-population.csv')
pop

,state/region,ages,year,population
0,AL,under18,2012,1117489.0
1,AL,total,2012,4817528.0
2,AL,under18,2010,1130966.0
3,AL,total,2010,4785570.0
4,AL,under18,2011,1125763.0
...,...,...,...,...
2539,USA,total,2010,309326295.0
2540,USA,under18,2011,73902222.0
2541,USA,total,2011,311582564.0
2542,USA,under18,2012,73708179.0


In [41]:
areas = pd.read_csv('data/state-areas.csv')
areas.head(8) # выводит первые 5 строк по defoult

,state,area (sq. mi)
0,Alabama,52423
1,Alaska,656425
2,Arizona,114006
3,Arkansas,53182
4,California,163707
5,Colorado,104100
6,Connecticut,5544
7,Delaware,1954


In [42]:
abbrevs = pd.read_csv('data/state-abbrevs.csv')
abbrevs.head()

,state,abbreviation
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA


In [43]:
print(pop.head(), areas.head(), abbrevs.head())  # криво

  state/region     ages  year  population
0           AL  under18  2012   1117489.0
1           AL    total  2012   4817528.0
2           AL  under18  2010   1130966.0
3           AL    total  2010   4785570.0
4           AL  under18  2011   1125763.0         state  area (sq. mi)
0     Alabama          52423
1      Alaska         656425
2     Arizona         114006
3    Arkansas          53182
4  California         163707         state abbreviation
0     Alabama           AL
1      Alaska           AK
2     Arizona           AZ
3    Arkansas           AR
4  California           CA


In [44]:
print(pop.head()); print(areas.head()); print(abbrevs.head())

  state/region     ages  year  population
0           AL  under18  2012   1117489.0
1           AL    total  2012   4817528.0
2           AL  under18  2010   1130966.0
3           AL    total  2010   4785570.0
4           AL  under18  2011   1125763.0
        state  area (sq. mi)
0     Alabama          52423
1      Alaska         656425
2     Arizona         114006
3    Arkansas          53182
4  California         163707
        state abbreviation
0     Alabama           AL
1      Alaska           AK
2     Arizona           AZ
3    Arkansas           AR
4  California           CA


In [46]:
merged = pd.merge(pop, abbrevs, how='outer',
                  left_on='state/region', right_on='abbreviation').drop('abbreviation', axis=1)
merged.head(9)

,state/region,ages,year,population,state
0,AL,under18,2012,1117489.0,Alabama
1,AL,total,2012,4817528.0,Alabama
2,AL,under18,2010,1130966.0,Alabama
3,AL,total,2010,4785570.0,Alabama
4,AL,under18,2011,1125763.0,Alabama
5,AL,total,2011,4801627.0,Alabama
6,AL,total,2009,4757938.0,Alabama
7,AL,under18,2009,1134192.0,Alabama
8,AL,under18,2013,1111481.0,Alabama


In [47]:
# проверка есть ли несовпадения
merged.isnull()

,state/region,ages,year,population,state
0,False,False,False,False,False
1,False,False,False,False,False
2,False,False,False,False,False
3,False,False,False,False,False
4,False,False,False,False,False
...,...,...,...,...,...
2539,False,False,False,False,True
2540,False,False,False,False,True
2541,False,False,False,False,True
2542,False,False,False,False,True


In [48]:
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state            True
dtype: bool

In [49]:
# выведем строки в котрых были пропуски по столбцу population
merged[merged['population'].isnull()]

,state/region,ages,year,population,state
2448,PR,under18,1990,NaN,NaN
2449,PR,total,1990,NaN,NaN
2450,PR,total,1991,NaN,NaN
2451,PR,under18,1991,NaN,NaN
2452,PR,total,1993,NaN,NaN
2453,PR,under18,1993,NaN,NaN
2454,PR,under18,1992,NaN,NaN
2455,PR,total,1992,NaN,NaN
2456,PR,under18,1994,NaN,NaN
2457,PR,total,1994,NaN,NaN


In [50]:
merged.loc[merged['state'].isnull()]

,state/region,ages,year,population,state
2448,PR,under18,1990,NaN,NaN
2449,PR,total,1990,NaN,NaN
2450,PR,total,1991,NaN,NaN
2451,PR,under18,1991,NaN,NaN
2452,PR,total,1993,NaN,NaN
...,...,...,...,...,...
2539,USA,total,2010,309326295.0,NaN
2540,USA,under18,2011,73902222.0,NaN
2541,USA,total,2011,311582564.0,NaN
2542,USA,under18,2012,73708179.0,NaN


In [51]:
merged.loc[merged['state'].isnull(), 'state/region'].unique()

array(['PR', 'USA'], dtype=object)

In [55]:
# исправим эти записи столбец state для записей в которых merged['state/region'] == 'PR'
merged.loc[merged['state/region'] == 'PR', 'state'] = 'Puerto Rico'

In [56]:
merged.loc[merged['state/region'] == 'USA', 'state'] = 'Unites States'

In [57]:
merged.loc[merged['state'].isnull()]

,state/region,ages,year,population,state


In [58]:
merged[merged['population'].isnull()]

,state/region,ages,year,population,state
2448,PR,under18,1990,NaN,Puerto Rico
2449,PR,total,1990,NaN,Puerto Rico
2450,PR,total,1991,NaN,Puerto Rico
2451,PR,under18,1991,NaN,Puerto Rico
2452,PR,total,1993,NaN,Puerto Rico
2453,PR,under18,1993,NaN,Puerto Rico
2454,PR,under18,1992,NaN,Puerto Rico
2455,PR,total,1992,NaN,Puerto Rico
2456,PR,under18,1994,NaN,Puerto Rico
2457,PR,total,1994,NaN,Puerto Rico


In [59]:
merged.head()

,state/region,ages,year,population,state
0,AL,under18,2012,1117489.0,Alabama
1,AL,total,2012,4817528.0,Alabama
2,AL,under18,2010,1130966.0,Alabama
3,AL,total,2010,4785570.0,Alabama
4,AL,under18,2011,1125763.0,Alabama


In [60]:
areas.head()

,state,area (sq. mi)
0,Alabama,52423
1,Alaska,656425
2,Arizona,114006
3,Arkansas,53182
4,California,163707


In [61]:
pd.merge(merged, areas, how='outer', left_on='state', right_on='state')

,state/region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,52423.0
1,AL,total,2012,4817528.0,Alabama,52423.0
2,AL,under18,2010,1130966.0,Alabama,52423.0
3,AL,total,2010,4785570.0,Alabama,52423.0
4,AL,under18,2011,1125763.0,Alabama,52423.0
...,...,...,...,...,...,...
2539,USA,total,2010,309326295.0,Unites States,NaN
2540,USA,under18,2011,73902222.0,Unites States,NaN
2541,USA,total,2011,311582564.0,Unites States,NaN
2542,USA,under18,2012,73708179.0,Unites States,NaN


In [75]:
final = pd.merge(merged, areas, how='left', on='state')
final

,state/region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,52423.0
1,AL,total,2012,4817528.0,Alabama,52423.0
2,AL,under18,2010,1130966.0,Alabama,52423.0
3,AL,total,2010,4785570.0,Alabama,52423.0
4,AL,under18,2011,1125763.0,Alabama,52423.0
...,...,...,...,...,...,...
2539,USA,total,2010,309326295.0,Unites States,NaN
2540,USA,under18,2011,73902222.0,Unites States,NaN
2541,USA,total,2011,311582564.0,Unites States,NaN
2542,USA,under18,2012,73708179.0,Unites States,NaN


In [76]:
final.isnull().any()

state/region     False
ages             False
year             False
population        True
state            False
area (sq. mi)     True
dtype: bool

In [77]:
# посмотрим какие территории не были учтены
final['state'][final['area (sq. mi)'].isnull()].unique()

array(['Unites States'], dtype=object)

In [78]:
# Удалим NaN
final.dropna(inplace=True)
final

,state/region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,52423.0
1,AL,total,2012,4817528.0,Alabama,52423.0
2,AL,under18,2010,1130966.0,Alabama,52423.0
3,AL,total,2010,4785570.0,Alabama,52423.0
4,AL,under18,2011,1125763.0,Alabama,52423.0
...,...,...,...,...,...,...
2491,PR,under18,2010,896945.0,Puerto Rico,3515.0
2492,PR,under18,2011,869327.0,Puerto Rico,3515.0
2493,PR,total,2011,3686580.0,Puerto Rico,3515.0
2494,PR,under18,2012,841740.0,Puerto Rico,3515.0


In [79]:
data2010 = final.query("year == 2010 & ages == 'total'")
data2010.head()

,state/region,ages,year,population,state,area (sq. mi)
3,AL,total,2010,4785570.0,Alabama,52423.0
91,AK,total,2010,713868.0,Alaska,656425.0
101,AZ,total,2010,6408790.0,Arizona,114006.0
189,AR,total,2010,2922280.0,Arkansas,53182.0
197,CA,total,2010,37333601.0,California,163707.0


In [80]:
# выполним переиндексацию
data2010.set_index('state', inplace=True)
data2010.head()
# после выполнения переиндексации, эту ячейку второй раз не выполнить, т.к. в буфере окажется уже переиндексированный DF

,state/region,ages,year,population,area (sq. mi)
state,,,,,
Alabama,AL,total,2010,4785570.0,52423.0
Alaska,AK,total,2010,713868.0,656425.0
Arizona,AZ,total,2010,6408790.0,114006.0
Arkansas,AR,total,2010,2922280.0,53182.0
California,CA,total,2010,37333601.0,163707.0


In [81]:
# вычислим плотность населения и выведем данные в соответствующем порядке
density = data2010['population'] / data2010['area (sq. mi)']

In [84]:
density.head()

state
Alabama        91.287603
Alaska          1.087509
Arizona        56.214497
Arkansas       54.948667
California    228.051342
dtype: float64

In [85]:
density.sort_values(ascending=False, inplace=True)
density.head()

state
District of Columbia    8898.897059
Puerto Rico             1058.665149
New Jersey              1009.253268
Rhode Island             681.339159
Connecticut              645.600649
dtype: float64

In [86]:
# хвост списка .tail()
density.tail()

state
South Dakota    10.583512
North Dakota     9.537565
Montana          6.736171
Wyoming          5.768079
Alaska           1.087509
dtype: float64